In [1]:
text = ["London Paris London","Paris Paris London"]

Now, we need to find a way to represent these texts as vectors. The `CountVectorizer()` class from `sklearn.feature_extraction.text` library can do this for us. We need to import this library before we can create a new `CountVectorizer()` object.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)

`count_matrix` gives us a sparse matrix. To make it in human readable form, we need to apply `toarrray()` method over it. And before printing out this `count_matrix`, let us first print out the feature list(or, word list), which have been fed to our `CountVectorizer()` object.

In [3]:
print(cv.get_feature_names())
print(count_matrix.toarray())

['london', 'paris']
[[2 1]
 [1 2]]


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


This indicates that the word ‘london’ occurs 2 times in A and 1 time in B. Similarly, the word ‘paris’ occurs 1 time in A and 2 times in B. Makes sense. Right?

Now, we need to find cosine(or “cos”) similarity between these vectors to find out how similar they are from each other. We can calculate this using `cosine_similarity()` function from `sklearn.metrics.pairwise` library.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(count_matrix)
print(similarity_scores)

[[1.  0.8]
 [0.8 1. ]]


What does this output indicate?

We can interpret this output like this-

1. Each row of the similarity matrix indicates each sentence of our input. So, row 0 = Text A and row 1 = Text B.
2. The same thing applies for columns. To get a better understanding over this, we can say that the output given above is same as the following:

<code>
        Text A:     Text B:
Text A: [[1.         0.8]  
Text B: [0.8         1.]]  
</code>
<br>
Interpreting this, says that Text A is similar to Text A(itself) by 100%(position [0,0]) and Text A is similar to Text B by 80%(position [0,1]). And by looking at the kind of output it is giving, we can easily say that this is always going to output a symmetric matrix. Because, if Text A is similar to Text B by 80% then, Text B is also going to be similar to Text A by 80%.


Now we know how to find similarity between contents. So, let’s try to apply this knowledge to build a content based movie recommendation engine.

### Building the recommendation engine:

>The movie dataset that we are going to use in our recommendation engine can be downloaded from [Course Github Repo](https://github.com/codeheroku/Introduction-to-Machine-Learning/blob/master/Building%20a%20Movie%20Recommendation%20Engine/movie_dataset.csv).

After downloading the dataset, we need to import all the required libraries and then read the csv file using `read_csv()` method.

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("/kaggle/input/movie-recommendation-system-sppu-2019/movie_dataset.csv")

If you visualize the dataset, you will see that it has many extra info about a movie. We don’t need all of them. So, we choose keywords, cast, genres and director column to use as our feature set(the so called “content” of the movie).

In [6]:
features = ['keywords','cast','genres','director']

Our next task is to create a function for combining the values of these columns into a single string.

In [7]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

Now, we need to call this function over each row of our dataframe. But, before doing that, we need to clean and preprocess the data for our use. We will fill all the NaN values with blank string in the dataframe.

In [8]:
for feature in features:
    df[feature] = df[feature].fillna('') #filling all NaNs with blank string

df["combined_features"] = df.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column

Now that we have obtained the combined strings, we can now feed these strings to a CountVectorizer() object for getting the count matrix.

In [9]:
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(df["combined_features"]) #feeding combined strings(movie contents) to CountVectorizer() object

At this point, 60% work is done. Now, we need to obtain the cosine similarity matrix from the count matrix.

In [10]:
cosine_sim = cosine_similarity(count_matrix)

Now, we will define two helper functions to get movie title from movie index and vice-versa.

In [11]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

Our next step is to get the title of the movie that the user currently likes. Then we will find the index of that movie. After that, we will access the row corresponding to this movie in the similarity matrix. Thus, we will get the similarity scores of all other movies from the current movie. Then we will enumerate through all the similarity scores of that movie to make a tuple of movie index and similarity score. This will convert a row of similarity scores like this- `[1 0.5 0.2 0.9]` to this- `[(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)]` . Here, each item is in this form- (movie index, similarity score).

In [12]:
movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it


Now comes the most vital point. We will sort the list `similar_movies` according to similarity scores in descending order. Since the most similar movie to a given movie will be itself, we will discard the first element after sorting the movies.

In [13]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:6]

Now, we will run a loop to print first 5 entries from `sorted_similar_movies` list.

In [14]:
i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Top 5 similar movies to Avatar are:

Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1
Star Trek Into Darkness
Star Trek Beyond


# Home Work Solution

Let's Inspect the vote_average feature and check if there are any null values. Looks like it is clean.

In [15]:
df["vote_average"].unique()

array([ 7.2,  6.9,  6.3,  7.6,  6.1,  5.9,  7.4,  7.3,  5.7,  5.4,  7. ,
        6.5,  6.4,  6.2,  7.1,  5.8,  6.6,  7.5,  5.5,  6.7,  6.8,  6. ,
        5.1,  7.8,  5.6,  5.2,  8.2,  7.7,  5.3,  8. ,  4.8,  4.9,  7.9,
        8.1,  4.7,  5. ,  4.2,  4.4,  4.1,  3.7,  3.6,  3. ,  3.9,  4.3,
        4.5,  3.4,  4.6,  8.3,  3.5,  4. ,  2.3,  3.2,  0. ,  3.8,  2.9,
        8.5,  1.9,  3.1,  3.3,  2.2,  0.5,  9.3,  8.4,  2.7, 10. ,  1. ,
        2. ,  2.8,  9.5,  2.6,  2.4])

Now, we will again sort our sorted_similar_movies but this time with respect to vote_average. x[0] has the index of the movie in the data frame.

In [16]:
sort_by_average_vote = sorted(sorted_similar_movies,key=lambda x:df["vote_average"][x[0]],reverse=True)
print(sort_by_average_vote)

[(3208, 0.3464101615137755), (94, 0.42339019740572564), (2403, 0.3774256780481986), (47, 0.34426518632954817), (56, 0.33596842045264647)]


In [17]:
i=0
print("Suggesting top 5 movies in order of Average Votes:\n")
for element in sort_by_average_vote:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Suggesting top 5 movies in order of Average Votes:

Star Wars: Clone Wars: Volume 1
Guardians of the Galaxy
Aliens
Star Trek Into Darkness
Star Trek Beyond
